# MCP(HTTP with SSE) を AutoGen の Selector Group Chat からコール

In [ ]:
import os
from dotenv import load_dotenv
from autogen_ext.tools.mcp import SseServerParams, mcp_server_tools
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination, TimeoutTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient, AzureOpenAIChatCompletionClient

load_dotenv()

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
apikey = os.getenv("AZURE_OPENAI_API_KEY")
model_deployment = os.getenv("AZURE_OPENAI_MODEL_DEPLOYMENT")


## HTTP with SSE 設定
SSE 転送では、サーバーは複数のクライアント接続を処理できる独立したプロセスとして動作します。

In [ ]:

# Setup server params for remote service
server_params = SseServerParams(url="http://localhost:8000/sse")

# Get all available tools
tools = await mcp_server_tools(server_params)

client = AzureOpenAIChatCompletionClient(
    azure_deployment=model_deployment,
    model="gpt-4o",
    api_key=apikey,
    api_version="2025-03-01-preview",
    azure_endpoint=endpoint,
)

# Define termination condition
max_msg_termination = MaxMessageTermination(max_messages=10)
text_termination = TextMentionTermination("TERMINATE")
time_terminarion = TimeoutTermination(120)
combined_termination = max_msg_termination | text_termination | time_terminarion


## Agent と Selector Team 定義

In [5]:
planning_agent = AssistantAgent(
    "PlanningAgent",
    description="タスクを計画するエージェント。新しいタスクが与えられたときに最初に起動するエージェントであるべきである。",
    model_client=client,
    system_message="""
    あなたはplanning agentです。
    あなたの仕事は、複雑なタスクをより小さく、管理しやすいサブタスクに分解することです。
    あなたのチームメンバーは次の通りです。
     - travel_agent: 日本の観光の専門家
    あなたは計画を立て、タスクを委任するだけで、自分で実行することはありません。

    タスクを割り当てる際には、このフォーマットを使用してください:
    1. <agent> : <task>

    最終回答が完成したら調査結果を要約し、文の最後に TERMINATE を含めること!
    """,
)
# Create an agent with all tools
travel_agent = AssistantAgent(
    name="TravelAgent",
    model_client=client,
    tools=tools,
    system_message="あなたは優秀なトラベルエージェントです。ツールを使って旅行の計画を手伝ってください。",
)

team = SelectorGroupChat(
    [planning_agent, travel_agent],
    model_client=client,
    termination_condition=combined_termination,
)


In [6]:

#task = "2025年おすすめの旅行先を推薦してください。" 
task = "なんかいい旅行先ない？"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
なんかいい旅行先ない？


/Users/han/opt/anaconda3/envs/autogen049/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_selector_group_chat.py:169: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(messages=select_speaker_messages)


---------- PlanningAgent ----------
1. travel_agent : 日本国内のおすすめ旅行先を調査して提案する

以上のタスクを割り当てます。


/Users/han/opt/anaconda3/envs/autogen049/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py:869: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


---------- TravelAgent ----------
[FunctionCall(id='call_QgLdka0fwYK9IC7II7NeuGc5', arguments='{}', name='suggest_onsen_destination')]
---------- TravelAgent ----------
[FunctionExecutionResult(content="[TextContent(type='text', text='城崎温泉（兵庫県）', annotations=None)]", name='suggest_onsen_destination', call_id='call_QgLdka0fwYK9IC7II7NeuGc5', is_error=False)]
---------- TravelAgent ----------
[TextContent(type='text', text='城崎温泉（兵庫県）', annotations=None)]
---------- PlanningAgent ----------
旅行のおすすめ先として「城崎温泉（兵庫県）」が提案されました。伝統的な温泉街の雰囲気と外湯巡りが楽しめる魅力的な場所です。TERMINATE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='なんかいい旅行先ない？', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=179, completion_tokens=31), metadata={}, content='1. travel_agent : 日本国内のおすすめ旅行先を調査して提案する\n\n以上のタスクを割り当てます。', type='TextMessage'), ToolCallRequestEvent(source='TravelAgent', models_usage=RequestUsage(prompt_tokens=280, completion_tokens=15), metadata={}, content=[FunctionCall(id='call_QgLdka0fwYK9IC7II7NeuGc5', arguments='{}', name='suggest_onsen_destination')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='TravelAgent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content="[TextContent(type='text', text='城崎温泉（兵庫県）', annotations=None)]", name='suggest_onsen_destination', call_id='call_QgLdka0fwYK9IC7II7NeuGc5', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='TravelAgent', models_usage=None, metadata={}, content="[TextConte